In [8]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

In [9]:
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_list_path_csv.json'

In [11]:
data = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        for path in reader[info] :
            data.append(path)

In [23]:
csv = data[2::3]


In [24]:
serie_one = data[0::3] 
serie_two = data[1::3]
number_serie = len(serie_one)
all_serie = []
for i in range(number_serie):
    serie = []
    serie.append(serie_one[i])
    serie.append(serie_two[i])
    all_serie.append(serie)


In [ ]:
for serie in all_serie : 
    for i in range(2):
        serie_object = Series(serie[i]) #objet série 
        #get numpy array
        #export nifti

In [ ]:
csv_directory = ''
for csv_file in csv : 
    mask = MaskBuilder(os.path.join(csv_directory, csv_file), size_matrix)
    #save mask builder
